In [11]:
%run Dataset_cleanup/cleanup_clinical.py

Clinical data set imported


 The following features do not provide any information: 
 ['Composite.Element.REF' 'ethnicity' 'gender' 'pathologicstage'
 'pathologyMstage' 'tumortissuesite'] 



 Variables that are not known at initial diagnosis: 
 ['daystodeath' 'daystolastfollowup' 'daystopsa' 'histologicaltype'
 'numberoflymphnodes' 'pathologyTstage' 'radiationtherapy' 'residualtumor'
 'vitalstatus'] 


Variables that are known at the time of diagnosis:
 ['dateofinitialpathologicdiagnosis' 'gleasonscore' 'psavalue' 'race'
 'yearstobirth']

Dimensions of clinical dataframe: (499, 8)


In [12]:
print(clinical.head())

                dateofinitialpathologicdiagnosis  gleasonscore  psavalue  \
clinical_index                                                             
TCGA-CH-5753                              2008.0           9.0      4.53   
TCGA-CH-5763                              2008.0           7.0      0.10   
TCGA-CH-5764                              2008.0           7.0     13.95   
TCGA-CH-5772                              2008.0           9.0      0.03   
TCGA-CH-5790                              2006.0           7.0      0.10   

                yearstobirth  race_Not_provided  race_asian  race_black_or_AA  \
clinical_index                                                                  
TCGA-CH-5753            70.0                0.0         0.0               0.0   
TCGA-CH-5763            66.0                0.0         0.0               0.0   
TCGA-CH-5764            66.0                0.0         0.0               0.0   
TCGA-CH-5772            63.0                0.0         0.0   

In [2]:
%run Dataset_cleanup/cleanup_gene_counts.py

Gene Counts data set imported

Dimension of DataFrame: (550, 20501)

Transformation Successful!

Transcript abundance estimates have been transformed to transcripts per million reads


## Analysis of initial clinical presentation for  predictors of metastasis

In [13]:
"""The Clinical data (including metastasis label) have a multitude of missing items."""
print("Total observations in original dataset:",clinical.shape[0])

not_labeled = y[y.isnull()]
y_labels = y[y.notnull()]

print("\nLabeled observations:",y_labels.shape[0],"\nUnlabeled observations removed:",not_labeled.shape[0])


Total observations in original dataset: 499

Labeled observations: 426 
Unlabeled observations removed: 73


In [16]:
clinical = clinical.loc[y_labels.index]
print(clinical.shape[0])

clinical.dropna(inplace = True)
print("Removing NaN rows leaves ",clinical.shape[0]," obersvations in the feature set.")

y_labels = y_labels.loc[clinical.index] #remove metastasis labels where no clinical data now exists due to NaN removal.

426
Removing NaN rows leaves  359  obersvations in the feature set.


In [21]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion='gini',
                            splitter='best',
                            max_depth=None,
                            min_samples_split=2,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0,
                            max_features=None,
                            random_state=None,
                            max_leaf_nodes=None,
                            class_weight='balanced',
                            presort=False)

clf.fit(clinical, y_labels)
print(pd.DataFrame({"Feature" : clinical.columns, "Gini_Importance": clf.feature_importances_} ))

                            Feature  Gini_Importance
0  dateofinitialpathologicdiagnosis         0.216585
1                      gleasonscore         0.274530
2                          psavalue         0.234825
3                      yearstobirth         0.215980
4                 race_Not_provided         0.018170
5                        race_asian         0.000000
6                  race_black_or_AA         0.017460
7                        race_white         0.022449


In [23]:
from sklearn.metrics import recall_score

In [26]:
recall_score(y_labels, clf.predict(clinical), labels=None, pos_label='n1', average='binary', sample_weight=None)

1.0